In [77]:
import pymysql

In [204]:
def connect():
    con = pymysql.connect(
        host = 'localhost',
        port = 3306,
        user = 'root',
        password = 'root',
        db = 'mldb3',
        charset = 'utf8'
    )
    print('1. 연결성공')
        
    cur = con.cursor()
    print('2. cursor 객체 획득')
    
    return cur, con

In [205]:
cur2, con = connect()
cur2

1. 연결성공
2. cursor 객체 획득


In [80]:
sql = """
    select o2.orderid, m.name, o2.productid, o2.totalprice
        from orderlist o2
            inner join `member` m
            on o2.memberid=m.id and m.id='park'
        order by productid desc
    """

In [81]:
result_count = cur2.execute(sql)
print('3. sql전송 성공', result_count)  # result 에는 count

3. sql전송 성공 6


In [82]:
rows = cur2.fetchall()
print('4. 검색결과 ----------------------')
print(rows)

4. 검색결과 ----------------------
((10, 'park', '103', 5000), (2, 'park', '101', 6000), (1, 'park', '100', 5000), (3, 'park', '100', 10000), (4, 'park', '100', 3000), (8, 'park', '100', 1000))


In [83]:
con.close()

In [84]:
cur2, con = connect()
cur2

1. 연결성공
2. cursor 객체 획득


In [85]:
for index, row in enumerate(rows):
    print(index, row)

0 (10, 'park', '103', 5000)
1 (2, 'park', '101', 6000)
2 (1, 'park', '100', 5000)
3 (3, 'park', '100', 10000)
4 (4, 'park', '100', 3000)
5 (8, 'park', '100', 1000)


In [86]:
def sql_execute(cur2, data):
    sql = """
        select o2.orderid, m.name, o2.productid, o2.totalprice
            from orderlist o2
                inner join `member` m
                on o2.memberid=m.id and m.id=%s
            order by productid desc
    """
    
    result = cur2.execute(sql, data)
    return result

In [87]:
sql_result = sql_execute(cur2, 'park')
sql_result

6

In [88]:
def out_data(cur2):
    rows = cur2.fetchall()
    return rows

In [89]:
result_rows = out_data(cur2)
result_rows

((10, 'park', '103', 5000),
 (2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000),
 (8, 'park', '100', 1000))

In [90]:
def db_close(con):
    con.close()

In [91]:
db_close(con)

In [92]:
## 제일 높은 금액은?

In [93]:
[x[3] for x in result_rows]

[5000, 6000, 5000, 10000, 3000, 1000]

In [94]:
max([x[3] for x in result_rows])

10000

In [95]:
## 제일 많이 산 물건의 id 는?

In [96]:
import pandas as pd

In [97]:
result_df = pd.DataFrame(data = result_rows, columns=['주문id','name','제품id','주문총가격'])

In [98]:
result_df

,주문id,name,제품id,주문총가격
0,10,park,103,5000
1,2,park,101,6000
2,1,park,100,5000
3,3,park,100,10000
4,4,park,100,3000
5,8,park,100,1000


In [99]:
[x for x in result_df['제품id']]

['103', '101', '100', '100', '100', '100']

In [100]:
from collections import Counter

In [101]:
counter_id = Counter(result_df['제품id'])

In [102]:
counter_id

Counter({'103': 1, '101': 1, '100': 4})

In [113]:
max_id = counter_id.most_common(n=3)
print(max_id)
print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

[('100', 4), ('103', 1), ('101', 1)]
100


'100'

In [104]:
def sql_execute2(cur2, data):
    sql="""
        select * from product
        where id = %s
    """
    result2 = cur2.execute(sql, data)
    return result2

In [105]:
cur2, con = connect()
cur2

1. 연결성공
2. cursor 객체 획득


In [106]:
sql_result2 = sql_execute2(cur2, max_id_result)
sql_result2

1

In [107]:
result_rows = out_data(cur2)
result_rows[0]

('100', 'java', 'fun java', '1000')

In [108]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('----------------------------------------------')
print('물건의 이름은 ', result_rows[0][1])
print('물건의 내용은 ', result_rows[0][2])
print('물건의 가격은 ', result_rows[0][3])

당신에게 추천하고 싶은 물건은 다음과 같습니다.
----------------------------------------------
물건의 이름은  java
물건의 내용은  fun java
물건의 가격은  1000


In [70]:
def sql_execute3(con, cur2, data):
    sql = """
        insert into orderlist values (null, %s, %s, %s)
    """
    result2 = cur2.execute(sql, data)
    con.commit()
    return result2

In [71]:
order_question = input("구매하시겠습니까?(yes/no)")
if order_question == 'yes':
    print('------------구매를 진행합니다.-----------------')
    your_id = input('당신의 id는 ')
    data_list = [your_id, result_rows[0][0], int(result_rows[0][3])]
    result3 = sql_execute3(con, cur2, data_list)
    if result3 == 1:
        print('------구매가 완료되었습니다.')
    else:
        print('------구매에 문제가 발생했습니다.')

구매하시겠습니까?(yes/no)yes
------------구매를 진행합니다.-----------------
당신의 id는 park
------구매가 완료되었습니다.


In [243]:
### 추천을 하고자 하는 컨텐츠기반 룰을 정하고,
### 물건을 3개 이상 추천하는 추천시스템을 완성하시오.
### db의 데이터를 임의로 추가할 수 있음.

In [259]:
cur2, con = connect()
cur2

1. 연결성공
2. cursor 객체 획득


In [260]:
from sqlalchemy import text

In [261]:
def sql_execute(cur2, data):
    
    print(data)
    sql = """
        select o2.memberid, p.content, o2.productid, o2.totalprice
            from product p
                inner join orderlist o2
                on p.id = o2.productid 
            where p.content like %s
        """
    
    result = cur2.execute(sql, data)
    return result

In [262]:
data = input('검색하고자 하는 물품명을 입력하세요')
sql_result = sql_execute(cur2, '%%%s%%' %data)
sql_result

검색하고자 하는 물품명을 입력하세요js
%js%


19

In [263]:
result_rows = out_data(cur2)
result_rows

(('park', 'fun jsp', '101', 6000),
 ('apple', 'fun jsp', '101', 2000),
 ('sun', 'fun jsp', '101', 4000),
 ('win', 'fun jsp', '101', 7000),
 ('win2', 'fun nodejs', '106', 3000),
 ('win3', 'fun nodejs', '106', 7000),
 ('park', 'nodejs nice', '111', 5000),
 ('apple', 'nodejs nice', '111', 7000),
 ('win3', 'nodejs nice', '111', 15000),
 ('park', 'javajsp nice', '112', 8000),
 ('apple', 'nodejsjsp good', '114', 13000),
 ('apple', 'nodejsjsp good', '114', 12000),
 ('win', 'nodejsjsp good', '114', 6000),
 ('win3', 'jspypoawo haha', '116', 15000),
 ('win3', 'jspypoawo haha', '116', 14000),
 ('apple', 'jspypoawo haha', '116', 11000),
 ('park', 'jsininini nice', '117', 12000),
 ('win', 'nodejs poor', '120', 5000),
 ('win2', 'nodejs very', '121', 4000))

In [264]:
min([x[3] for x in result_rows])

2000

In [265]:
result_df = pd.DataFrame(data = result_rows, columns=['name','content','제품id','주문총가격'])
result_df

,name,content,제품id,주문총가격
0,park,fun jsp,101,6000
1,apple,fun jsp,101,2000
2,sun,fun jsp,101,4000
3,win,fun jsp,101,7000
4,win2,fun nodejs,106,3000
5,win3,fun nodejs,106,7000
6,park,nodejs nice,111,5000
7,apple,nodejs nice,111,7000
8,win3,nodejs nice,111,15000
9,park,javajsp nice,112,8000


In [266]:
[x for x in result_df['제품id']]

['101',
 '101',
 '101',
 '101',
 '106',
 '106',
 '111',
 '111',
 '111',
 '112',
 '114',
 '114',
 '114',
 '116',
 '116',
 '116',
 '117',
 '120',
 '121']

In [267]:
counter_id = Counter(result_df['제품id'])
counter_id

Counter({'101': 4,
         '106': 2,
         '111': 3,
         '112': 1,
         '114': 3,
         '116': 3,
         '117': 1,
         '120': 1,
         '121': 1})

In [268]:
max_id = counter_id.most_common(n=3)
print(max_id)
print(max_id[0][0])
max_id_result = [max_id[0][0], max_id[1][0], max_id[2][0]]
max_id_result

[('101', 4), ('111', 3), ('114', 3)]
101


['101', '111', '114']

In [269]:
len(max_id_result)

3

In [270]:
result_rows = out_data(cur2)
result_rows[0]

IndexError: tuple index out of range

In [256]:
result_rows = out_data(cur2)
for i in range(len(max_id_result)):
    print(result_rows[i])

IndexError: tuple index out of range